# 家計調査データ視覚化

## 出典

総務省統計局ホームページ

家計調査（家計収支編）　時系列データ（二人以上の世帯）

https://www.stat.go.jp/data/kakei/longtime/index.html#time

- 月 全品目（2020年改定）

（2024年6月3日に利用）

## 準備

必要な Python モジュール（プログラム部品）のインストール、読込

In [ ]:
# グラフで日本語を表示するためのモジュールをインストールする
!pip install japanize-matplotlib

In [ ]:
# 必要なモジュールを読み込む
import io

import japanize_matplotlib
import matplotlib.dates as mdates
import pandas as pd
import requests
import seaborn as sns

from matplotlib import pyplot as plt

## データダウンロード

URL を指定してダウンロードする

直接ブラウザからURLにアクセスするとCSVファイルをダウンロード・表示できる

In [ ]:
DATA_URL = 'https://www.stat.go.jp/data/kakei/longtime/csv/h-mon-2020.csv'
response = requests.get(DATA_URL)

ダウンロードしたデータはバイナリ（2進数）形式になっているため、そのまま表示しても意味が分からない

In [ ]:
response.content

「ShiftJIS」の文字コードで変換すると、文字として読むことができるようになる

In [ ]:
raw_data = response.content.decode('sjis')
raw_data

`pd.read_csv` でデータを構造化して読み込むことができる

ただし、そのままだと先頭の3行が邪魔で、正しく構造化されません

In [ ]:
# データフレームという形式で読み込む
df = pd.read_csv(io.StringIO(raw_data))
df

## データ加工

先頭3行を消します

In [ ]:
# '\r\n' は改行を意味するコード
# 改行で区切る（split）ことで、文字列を行毎に分割する
rows = raw_data.split('\r\n')
rows

In [ ]:
# [3:]で範囲を指定している
# Python では 0, 1, 2, 3 ... と順序を指定するため、 [3:] は4行目以降という意味になる
essential_rows = rows[3:]
essential_rows

先頭行が「列名」として使えるものになりました
ただし、「1月」「2月」「3月」などの列名に年が含まれていないため、加工します

In [ ]:
# [0]で先頭行だけを取得する
header = essential_rows[0]
header

In [ ]:
# 先頭行を`,`(カンマ)で区切り、各列名を処理する
new_cols = []
key_cols = []
month_cols = []
for (index, col) in enumerate(header.split(',')):
  if index <= 7:
    new_cols.append(col)
    key_cols.append(col)
  else:
    month = (index - 8) % 12 + 1
    year = (index - 8) // 12 + 2020
    col = f"month{year}{('0' + str(month))[-2:]}01"
    new_cols.append(col)
    month_cols.append(col)

new_header = ','.join(new_cols)
new_header

In [ ]:
# 先頭行を変換後の文字列に入れ替える
essential_rows[0] = new_header
essential_rows

加工したデータを構造化することで、表として見ることができるようになりました

例えば2020年1月、1世帯当たり784円食パンに消費していることが分かります

In [ ]:
# 全ての行を改めて結合し、データフレームに読み込む
df = pd.read_csv(io.StringIO('\r\n'.join(essential_rows)))
df

時間による推移を見たいため、年月を「列」ではなく「行」にします

このように「列」と「行」を入れ替える = 軸を回転することをピボットといいます

ピボットにより、品目分類と年月を指定して検索するようなことがしやすくなりました

In [ ]:
# `wide_to_long` で横長（wide）になっているデータを縦長(long)にする
pivot_df = pd.wide_to_long(df,stubnames=['month'],i='品目分類',j='yyyymmdd').reset_index()
pivot_df['yyyymmdd'] = pd.to_datetime(pivot_df['yyyymmdd'].astype('str'))
pivot_df = pivot_df[key_cols + ['yyyymmdd', 'month']].rename(columns={'yyyymmdd': '年月', 'month': '支出額'})
pivot_df

## データ抽出

以下のように指定することで、ケーキの支出額推移を見ることができます

In [ ]:
pivot_df[pivot_df['品目分類'] == 'ケーキ']

## データ集計

以下のようにすることでケーキの支出額について、基本統計量を確認することができます

各行の意味は以下の通りです

- mean: 平均
- min: 最小値
- 25%: 第１四分位数
- 50%: 中央値
- 75%: 第3四分位数
- max: 最大値
- std: 標準偏差

各列について見てみましょう

- 「表側連番」は名義尺度なので基本統計量に意味がありません
- 「年月」は間隔尺度です。このデータの時間的な範囲を確認できます。
- 「支出額」は比例尺度です。おおよそどのように分布しているのか見ることができます。

In [ ]:
pivot_df[pivot_df['品目分類'] == 'ケーキ'].describe()

## データの視覚化

以下のようにして支出額の箱ひげ図が作成できます

中央値「564円」の近辺に箱、ひげが描画され、1400近辺に外れ値があることが分かります

In [ ]:
fig, ax = plt.subplots()
ax.boxplot(pivot_df[pivot_df['品目分類'] == 'ケーキ']['支出額'])
plt.show()

以下のようにして時系列のグラフを作成できます

ケーキの支出額が毎年12月に突出していることが分かります

ここから、クリスマスに圧倒的にケーキが多く買われていることが推測できます

In [ ]:
# 時系列グラフを出力するための関数（処理のかたまり）
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['年月']
  ys = series['支出額']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5.2), layout='constrained')

# 毎月ラベルを表示する
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
ax.xaxis.set_major_locator(mdates.MonthLocator())

# 年月で並び替えて時系列にする
df_sorted = pivot_df[pivot_df['品目分類'] == 'ケーキ'].sort_values('年月', ascending=True)

# 時系列グラフを出力する
_plot_series(df_sorted, '')

sns.despine(fig=fig, ax=ax)
plt.xlabel('年月')
_ = plt.ylabel('支出額')

# ラベルを垂直にする
fig.autofmt_xdate(rotation=90, ha="center")